<a href="https://colab.research.google.com/github/23011985uhi/23011985_DataAnalytics/blob/main/DNNAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Neural Network

The aim is to produce an accurate DNN model that could learn from a training set of data and be able to predict the number of collisions if shown another set of test data. The training and test data have been prepared before using one hot encoding to create many columns incorporating month and day. This is a method of counting unique categories such as a day of the week or what month since there is only a yes/no answer for each state.

In [196]:
import pandas as pd

In [197]:
dff=pd.read_csv('https://raw.githubusercontent.com/23011985uhi/23011985_DataAnalytics/main/dnndata_nostandard_999removed.csv')

In [198]:
import numpy as np

In [199]:
print(dff)

      Apr  Aug  Dec  Feb  Jan  Jul  Jun  Mar  May  Nov  ...  temp  dewp  \
0       0    0    0    0    1    0    0    0    0    0  ...  37.8  23.6   
1       0    0    0    0    1    0    0    0    0    0  ...  27.1  10.5   
2       0    0    0    0    1    0    0    0    0    0  ...  28.4  14.1   
3       0    0    0    0    1    0    0    0    0    0  ...  33.4  18.6   
4       0    0    0    0    1    0    0    0    0    0  ...  36.1  18.7   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
2520    0    0    1    0    0    0    0    0    0    0  ...  49.4  40.9   
2521    0    0    1    0    0    0    0    0    0    0  ...  48.0  37.4   
2522    0    0    1    0    0    0    0    0    0    0  ...  42.6  30.2   
2523    0    0    1    0    0    0    0    0    0    0  ...  39.4  38.3   
2524    0    0    1    0    0    0    0    0    0    0  ...  38.7  34.9   

      num_collisions     slp  visib  wdsp  gust  prcp  sndp  fog  
0                381  1011.9   1

The data is imported in and shows many more variables than the Linear regressor data. A DNN is working with complex relationships rather than linear ones and so more data and variables may be required to reach an accurate result.

In [200]:
df = dff.copy()
df["num_collisions"] /= 1000


The crude scaling method from the Linear methods is being used again since it produced the better models previously. It is simply scaling num_collisions down to a number between 0-1 by dividing by 1000.

In [201]:
print(df)

      Apr  Aug  Dec  Feb  Jan  Jul  Jun  Mar  May  Nov  ...  temp  dewp  \
0       0    0    0    0    1    0    0    0    0    0  ...  37.8  23.6   
1       0    0    0    0    1    0    0    0    0    0  ...  27.1  10.5   
2       0    0    0    0    1    0    0    0    0    0  ...  28.4  14.1   
3       0    0    0    0    1    0    0    0    0    0  ...  33.4  18.6   
4       0    0    0    0    1    0    0    0    0    0  ...  36.1  18.7   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
2520    0    0    1    0    0    0    0    0    0    0  ...  49.4  40.9   
2521    0    0    1    0    0    0    0    0    0    0  ...  48.0  37.4   
2522    0    0    1    0    0    0    0    0    0    0  ...  42.6  30.2   
2523    0    0    1    0    0    0    0    0    0    0  ...  39.4  38.3   
2524    0    0    1    0    0    0    0    0    0    0  ...  38.7  34.9   

      num_collisions     slp  visib  wdsp  gust  prcp  sndp  fog  
0              0.381  1011.9   1

In [202]:

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.15.0


In [203]:
dnn_input_data = [df["year"], df["temp"], df["dewp"], df["num_collisions"], df["Sat"], df["Sun"], df["Mon"], df["Tue"], df["Wed"], df["Thu"], df["Fri"], df["Jan"], df["Feb"], df["Mar"], df["Apr"], df["May"], df["Jun"], df["Jul"], df["Aug"], df["Sep"], df["Oct"], df["Nov"], df["Dec"], df["slp"], df["visib"], df["wdsp"], df["gust"], df["prcp"], df["sndp"], df["fog"]]
headers = ["year","temp", "dewp", "num_collisions", "Sat","Sun","Mon","Tue","Wed","Thu","Fri","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec","slp","visib","wdsp","gust","prcp","sndp","fog"]
df_dnn_input = pd.concat(dnn_input_data, axis=1, keys=headers)
df_dnn_input.head()

,year,temp,dewp,num_collisions,Sat,Sun,Mon,Tue,Wed,Thu,...,Oct,Nov,Dec,slp,visib,wdsp,gust,prcp,sndp,fog
0,2013,37.8,23.6,0.381,0,0,0,1,0,0,...,0,0,0,1011.9,10.0,6.1,19.0,0.0,0.0,0
1,2013,27.1,10.5,0.480,0,0,0,0,1,0,...,0,0,0,1016.8,10.0,5.3,19.0,0.0,0.0,0
2,2013,28.4,14.1,0.549,0,0,0,0,0,1,...,0,0,0,1020.6,10.0,3.7,15.0,0.0,0.0,0
3,2013,33.4,18.6,0.505,0,0,0,0,0,0,...,0,0,0,1017.0,10.0,6.5,24.1,0.0,0.0,0
4,2013,36.1,18.7,0.389,1,0,0,0,0,0,...,0,0,0,1020.6,10.0,6.6,21.0,0.0,0.0,0


This is the datafram formatting and creation. The test dataset used later will need to have the same format and layout or the testing process will not work.

In [204]:
training_dataset = df_dnn_input.sample(frac=0.8, random_state=0)
test_dataset = df_dnn_input.drop(training_dataset.index)

The model is trained on 80% of the data and tested on 20% , like the linear models previously. A DNN is similar to a linear model but has more layers.

In [205]:
training_features = training_dataset.copy()
test_features = test_dataset.copy()

training_labels = training_features.pop('num_collisions')
test_labels = test_features.pop('num_collisions')

In [206]:
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features))

In [207]:
dnn_model_1 = keras.Sequential([
      normaliser,
      layers.Dense(48, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
  ])

dnn_model_1.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

This is the same process as the linear models except you can see above how there are 48 layers compared to the linear of 1.

In [208]:
%%time
history = dnn_model_1.fit(
    training_features,
    training_labels,
    validation_split=0.2,
    verbose=0,
    epochs=100)

CPU times: user 14.9 s, sys: 678 ms, total: 15.6 s
Wall time: 21.2 s


The training features and training labels are fit to the model and it is run 100 times with another 20% validation.

In [209]:
dnn_model_1_results = dnn_model_1.evaluate(test_features, test_labels, verbose=0)
print(dnn_model_1_results)

0.06483278423547745


The results are quite promising and show a very low value. The closer to 0 should be a better and more accurate model. Just need to test the model to see if it can predict accurately or not.

In [210]:
testdf = pd.read_csv('https://raw.githubusercontent.com/23011985uhi/23011985_DataAnalytics/main/dnndatatestgood.csv', index_col=0)

In [211]:
testdf.replace(999.9, 0, inplace=True)

The test data had some unknown values that were being displayed as 999.9 in the dataset. This code just takes all the 999.9 unknown values and sets them to 0. It could effect the results but it should not be much as its just replacing one constant for another.

In [212]:
testdf = [testdf["year"], testdf["temp"], testdf["dewp"], testdf["num_collisions"], testdf["Sat"], testdf["Sun"], testdf["Mon"], testdf["Tue"], testdf["Wed"], testdf["Thu"], testdf["Fri"], testdf["Jan"], testdf["Feb"], testdf["Mar"], testdf["Apr"], testdf["May"], testdf["Jun"], testdf["Jul"], testdf["Aug"], testdf["Sep"], testdf["Oct"], testdf["Nov"], testdf["Dec"], testdf["slp"], testdf["visib"], testdf["wdsp"], testdf["gust"], testdf["prcp"], testdf["sndp"], testdf["fog"]]
headers = ["year","temp", "dewp", "num_collisions", "Sat","Sun","Mon","Tue","Wed","Thu","Fri","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec","slp","visib","wdsp","gust","prcp","sndp","fog"]
testdf = pd.concat(testdf, axis=1, keys=headers)
testdf.head()

,year,temp,dewp,num_collisions,Sat,Sun,Mon,Tue,Wed,Thu,...,Oct,Nov,Dec,slp,visib,wdsp,gust,prcp,sndp,fog
1,2020,39.0,23.9,346,0,0,1,0,0,0,...,0,0,0,1006.0,10.0,7.9,21.0,0.02,0.0,0
2,2020,39.6,22.4,413,0,0,0,1,0,0,...,0,0,0,1013.2,10.0,5.2,17.1,0.00,0.0,0
3,2020,46.6,38.6,456,0,0,0,0,1,0,...,0,0,0,1009.7,8.1,3.7,15.0,0.06,0.0,0
4,2020,48.6,46.2,353,0,0,0,0,0,1,...,0,0,0,1005.7,4.9,2.8,0.0,0.34,0.0,1
5,2020,40.2,24.7,291,0,0,0,0,0,0,...,0,0,0,1007.9,10.0,10.5,34.0,0.02,0.0,0


In [213]:
print(testdf)

      year  temp  dewp  num_collisions  Sat  Sun  Mon  Tue  Wed  Thu  ...  \
1     2020  39.0  23.9             346    0    0    1    0    0    0  ...   
2     2020  39.6  22.4             413    0    0    0    1    0    0  ...   
3     2020  46.6  38.6             456    0    0    0    0    1    0  ...   
4     2020  48.6  46.2             353    0    0    0    0    0    1  ...   
5     2020  40.2  24.7             291    0    0    0    0    0    0  ...   
...    ...   ...   ...             ...  ...  ...  ...  ...  ...  ...  ...   
1092  2022  30.5  11.3             213    0    1    0    0    0    0  ...   
1093  2022  38.3  20.8             221    0    0    1    0    0    0  ...   
1094  2022  44.9  24.6             203    0    0    0    1    0    0  ...   
1095  2022  51.9  31.1             299    0    0    0    0    1    0  ...   
1096  2022  52.2  47.0             249    0    0    0    0    0    1  ...   

      Oct  Nov  Dec     slp  visib  wdsp  gust  prcp  sndp  fog  
1       0

In [214]:
testdf["num_collisions"] /= 1000

The same scaling process to get num_collisions to a number between 0 and 1.

In [215]:
test_features1 = testdf[headers].copy()
test_labels1 = test_features1.pop('num_collisions')

In [216]:
test_features_normalized = normaliser(test_features1)


predictions = dnn_model_1.predict(test_features_normalized)




comparison_df = pd.DataFrame({
    'Actual': test_labels1.values,
    'Predicted': predictions.flatten()
})


35/35 [==============================] - 0s 1ms/step


The test data fed through the model and then a seprate dataframe is created to compare the predictions to the actual values in the test dataset.

In [217]:
print(comparison_df.head())

   Actual  Predicted
0   0.346 -21.842058
1   0.413 -21.385546
2   0.456 -21.919184
3   0.353 -22.511644
4   0.291 -21.770197


The results are not very good looking at all in comparison to the model itself. Testing seems to have resulted in some bad predictions. Since the predictions are producing negative values it was be best to assume that the test data is influencing this. The model itself produced a good results and was looking fairly reliable but maybe there is an issue with the test data for this dnn.

# Conclusion

The DNN model that was created has produced good results which show great potential for further uses. It seems that the training data used may not be appropriate for use with the model and resulted in negative predictions. A proper analysis of the testing data would be the next step and possibly creation of new sets of test data to re-test the model. After that it would become clear as whether the model itself is the issue or it is faulty test data.